# Medicane Zorbas (september 2018) : ECMWF ensemble forecast

*Author : Frédéric FERRY - Météo-France / Ecole Nationale de la Météorologie (June 2023)*

Python concepts :
- Open netcdf files (xarray)
- Create maps (matplotlib, cartopy)

Advanced concepts :
- Perform EOF analysis of ensemble forecasts (eofs package)
- Perform k-means clustering (sklearn package)

In [ ]:
import os
import numpy as np

import xarray as xr

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
import matplotlib.patches as mpatches

import metpy.calc as mpcalc
from metpy.units import units

from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

from eofs.standard import Eof
from sklearn.cluster import KMeans
import scipy.spatial.distance as sdist

from tqdm import tqdm

# Part 1 : open ECMWF ensemble forecasts

In [ ]:
EPS_Z500 = './data/EPS_2018092400_Z500.nc'
EPS_T500 = './data/EPS_2018092400_T500.nc'
EPS_MSLP = './data/EPS_2018092400_MSLP.nc'
EPS_PV320 = './data/EPS_2018092400_PV320.nc'
EPS_qT850 = './data/EPS_2018092400_qT850.nc'

z500_eps    = xr.open_dataset(EPS_Z500)
t500_eps    = xr.open_dataset(EPS_T500)
mslp_eps   = xr.open_dataset(EPS_MSLP)
pv320_eps   = xr.open_dataset(EPS_PV320)
qT850_eps   = xr.open_dataset(EPS_qT850)

print(z500_eps)
print(t500_eps)
print(mslp_eps)
print(pv320_eps)
print(qT850_eps)

members = np.linspace(1, 50, 50)
members_str =[str(i) for i in range(1,51)]

<div class="alert alert-danger">
<p><b>Reduce the geographical domain to 25N-50N 0W-30E.</b></p>
</div>

In [ ]:
latS = 
latN = 
lonW = 
lonE = 

In [ ]:
print("Available forecast dates for the 2018-09-24 0000 UTC run: ")
print(z500_eps.time.values)
date = input("Enter the forecast date around the expeted 'medicane' time of the YYYY-MM-DDTXX : ")
date_str=date[0:13]
print(date_str)

In [ ]:
z500_eps    = z500_eps.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE)).sel(time=date)
t500_eps    = t500_eps.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE)).sel(time=date)
mslp_eps    = mslp_eps.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE)).sel(time=date)
pv320_eps    = pv320_eps.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE)).sel(time=date)
qT850_eps    = qT850_eps.sel(latitude=slice(latN,latS)).sel(longitude=slice(lonW,lonE)).sel(time=date)

lat = z500_eps.latitude.values
lon = z500_eps.longitude.values

In [ ]:
z500_ens=z500_eps['z']/9.81
t500_ens=t500_eps['t']-273.15
mslp_ens=mslp_eps['msl']/100
pv320_ens=pv320_eps['pv']*1e6
q850_ens=qT850_eps['q']
t850_ens=qT850_eps['t']

print(z500_ens)
print(t500_ens)
print(mslp_ens)
print(pv320_ens)
print(q850_ens)
print(t850_ens)

In [ ]:
Td = mpcalc.dewpoint_from_specific_humidity(850* units.hPa, t850_ens.values* units.kelvin, q850_ens.values* units('kg/kg'))
Thetae_ens=mpcalc.equivalent_potential_temperature(850* units.hPa, t850_ens, Td)
print(Thetae_ens.shape)
print(np.min(Thetae_ens))
print(np.max(Thetae_ens))

In [ ]:
clevs_z = np.linspace(5500, 6000, 21) # z500
clevs_mslp = np.arange(994, 1040, 2) # MSLP
clevs_pv = np.linspace(1, 6, 11) # PV320K
clevs_pv2 = np.linspace(1, 4, 11) # PV320K
clevs_the = np.arange(270,345,5) #thetae850
clevs_t500=np.arange(-26,-4,2) # t500
clevs_t850=np.arange(280,302,2) # t850

cmap='jet'

In [ ]:
projection = ccrs.PlateCarree()

def plot_background(ax):
    ax.coastlines()
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    ax.set_xticks(np.linspace(-180, 180, 19), crs=projection)
    ax.set_yticks(np.linspace(-90, 90, 19), crs=projection)
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    return ax

# Part 2 : ensemble mean

<div class="alert alert-danger">
<p><b>Compute the ensemble means for the variables Z500, T500, MSLP, PV320K and Thetae850 using the mean method of xarray : 

https://docs.xarray.dev/en/stable/generated/xarray.DataArray.mean.html.
</b></p>
</div>

In [ ]:
print(z500_ens)
z500_mean = 
t500_mean = 
mslp_mean = 
pv320_mean = 
the_mean = 

In [ ]:
fig = plt.figure(figsize=(15, 5))
fig.suptitle('Ensemble mean : '+date_str, fontsize=14)

ax = fig.add_subplot(131, projection=ccrs.PlateCarree())
plt.title('ZT500 ensemble mean', fontsize=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, t500_mean, levels=clevs_t500, cmap=cmap, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, z500_mean, levels=clevs_z, colors='black', linewidths=0.5, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('°C', fontsize=12)

ax = fig.add_subplot(132, projection=ccrs.PlateCarree())
plt.title('MSLP-THE850 ensemble mean', fontsize=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, the_mean, levels=clevs_the, cmap=cmap, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, mslp_mean, levels=clevs_mslp, colors='black', linewidths=0.5, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('K', fontsize=12)

ax = fig.add_subplot(133, projection=ccrs.PlateCarree())
plt.title('PV320K ensemble mean', fontsize=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, pv320_mean, levels=clevs_pv2, cmap=cmap, extend='max', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, pv320_mean, levels=clevs_pv2, colors='black', linewidths=0.5, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('PVU', fontsize=12)

plt.show()

figname='./figs/ZT500_MSLP_THE_PV320_ENS_MEAN_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Do the ensemble mean fields indicate favorable conditions for the formation of a medicane? </p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

# Part 3 : ensemble spread

<div class="alert alert-danger">
<p><b>Compute the ensemble spread (i.e standard deviation of the ensemble members) for the variables Z500 and MSLP using the std method of xarray :
    
https://docs.xarray.dev/en/stable/generated/xarray.DataArray.std.html
</b></p>
</div>

In [ ]:
print(z500_ens)
z_spread = 
mslp_spread = 

In [ ]:
clevs_spread1 = np.arange(10, 70, 5) # z500 spread

fig = plt.figure(figsize=(10, 10))
fig.suptitle('Geopotential height at 500hPa : ensemble spread', fontsize=14)
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
plt.title('Forecast time : '+date_str,size=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, z_spread, clevs_spread1, transform=ccrs.PlateCarree(), cmap=cmap, extend='both')
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('mgp', fontsize=12)
plt.show()

figname='./figs/Z500_ENS_SPREAD_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Locate the area with the maximum spread for the variable Z500. What does this spread indicates ?</p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

In [ ]:
clevs_spread2 = np.arange(1, 6.5, 0.5) # z500 spread

fig = plt.figure(figsize=(10, 10))
fig.suptitle('Mean Sea Level Pressure : ensemble spread', fontsize=14)
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
plt.title('Forecast time : '+date_str,size=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, mslp_spread, clevs_spread2, transform=ccrs.PlateCarree(), cmap=cmap, extend='both')
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('hPa', fontsize=12)
plt.show()

figname='./figs/MSLP_ENS_SPREAD_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Locate the area with the maximum spread for the variable MSLP. What does this spread indicates ?</p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

# Part 4 : spaguetti plots

In [ ]:
z_number=int(input("Select a suitable contour value (in meters) to isolate the Z500 low in the region of maximum spread : "))

clevs_spag = np.arange(z_number, z_number+1, 1) # 5640

fig = plt.figure(figsize=(15, 10))
fig.suptitle('Geopotential height at 500hPa spaguettis : '+ str(clevs_spag)+' mgp contour', fontsize=14)
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
plt.title('Forecast time : '+date_str,size=10, loc='center')
plot_background(ax)
for i in range(len(members)):
    c = ax.contour(lon, lat, z500_ens[i,:,:], clevs_spag, colors='blue', linewidths=1, transform=ccrs.PlateCarree())
plt.show()

figname='./figs/Z500_ENS_SPAG_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
msl_number=int(input("Select a suitable contour value to isolate the medicane in the region of maximum spread : "))

clevs_spag2 = np.arange(msl_number, msl_number+1, 1) # 1005

fig = plt.figure(figsize=(15, 10))
fig.suptitle('Mean Sea Level Pressure spaguettis : '+ str(clevs_spag2)+' hPa contour', fontsize=14)
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
plt.title('Forecast time : '+date_str,size=10, loc='center')
plot_background(ax)
for i in range(len(members)):
    c = ax.contour(lon, lat, mslp_ens[i,:,:], clevs_spag2, colors='blue', linewidths=1, transform=ccrs.PlateCarree())
plt.show()

figname='./figs/MSLP_ENS_SPAG_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) How many ensemble members seem to simulate a deep low pressure system ?</b> </p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

# Part 5 : stamp plots

In [ ]:
plt_title = 'ECMWF ensemble members - ZT500. Forecast time : '+date_str
projection = ccrs.PlateCarree()
#axes_class = (GeoAxes, dict(projection=projection)) # latest matplotlib
axes_class = (GeoAxes, dict(map_projection=projection))

fig = plt.figure(figsize=(20,20))
fig.suptitle(plt_title, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(10, 5),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='1%',
       label_mode='keep')
                   
for i, ax in enumerate(axgr):
    plot_background(ax)
    ax.set_title(str(i+1), fontsize=12)
    p1 = ax.contourf(lon, lat, t500_ens[i,:,:], clevs_t500, transform=ccrs.PlateCarree(), cmap=cmap, extend='both')
    p2 = ax.contour(lon, lat, z500_ens[i,:,:], clevs_z, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(p1)

plt.show()

figname='./figs/ZT500_ENS_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
plt_title = 'ECMWF ensemble members - MSLP and $\Theta_E$ at 850 hPa. Forecast time : '+date_str
projection = ccrs.PlateCarree()

fig = plt.figure(figsize=(20,20))
fig.suptitle(plt_title, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(10, 5),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='1%',
       label_mode='keep')
                   
for i, ax in enumerate(axgr):
    plot_background(ax)
    ax.set_title(str(i+1), fontsize=12)
    p1 = ax.contourf(lon, lat, Thetae_ens[i,:,:], clevs_the, transform=ccrs.PlateCarree(), cmap=cmap, extend='both')
    p2 = ax.contour(lon, lat, mslp_ens[i,:,:], clevs_mslp, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(p1)

plt.show()

figname='./figs/MSLP_Thetae850_ENS_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
plt_title = 'ECMWF ensemble members - PV320K. Forecast time : '+date_str
projection = ccrs.PlateCarree()

fig = plt.figure(figsize=(20,20))
fig.suptitle(plt_title, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(10, 5),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='1%',
       label_mode='keep')
                   
for i, ax in enumerate(axgr):
    plot_background(ax)
    ax.set_title(str(i+1), fontsize=12)
    p1 = ax.contourf(lon, lat, pv320_ens[i,:,:], clevs_pv, transform=ccrs.PlateCarree(), cmap=cmap, extend='max')
    axgr.cbar_axes[i].colorbar(p1)

plt.show()

figname='./figs/PV320_ENS_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Look at the different stamps and try to access the uncertainty of the forecast. How many scenarios seem to emerge in the ensemble ?</p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

# Part 6 (advanced) : EOF analysis of the ensemble

In [ ]:
wgts = np.sqrt(np.cos(np.deg2rad(lat)))[:, np.newaxis]
tab=[z500_ens, t500_ens, mslp_ens, pv320_ens, t850_ens]
variables=["Z500", "T500", "MSLP", "PV320K", "T850"]

clevs = np.linspace(-0.01, 0.01, 11)

i=0
for var in tab :
    data = var
    data = np.array(data)
    solver = Eof(data, weights=wgts)
    varfrac = solver.varianceFraction()
    eofs = solver.eofs()

    fig = plt.figure(figsize=(15, 8))
    fig.suptitle('ECMWF ensemble members - EOF analysis : '+variables[i]+' - '+date_str, fontsize=16)
    ax = fig.add_subplot(121, projection=ccrs.PlateCarree())
    varfrac1=round(varfrac[0]*100,2)
    plt.title('EOF1 (explained variance : '+str(varfrac1)+'%)', fontsize=10, loc='center')
    ax.coastlines()
    cf = ax.contourf(lon, lat, eofs[0], clevs, cmap='bwr', extend='both', transform=ccrs.PlateCarree())
    c = ax.contour(lon, lat, eofs[0], clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.7, pad=0.05, extendrect='True')

    ax = fig.add_subplot(122, projection=ccrs.PlateCarree())
    varfrac1=round(varfrac[1]*100,2)
    plt.title('EOF1 (explained variance : '+str(varfrac1)+'%)', fontsize=10, loc='center')
    ax.coastlines()
    cf = ax.contourf(lon, lat, eofs[1], clevs, cmap='bwr', extend='both', transform=ccrs.PlateCarree())
    c = ax.contour(lon, lat, eofs[1], clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
    cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.7, pad=0.05, extendrect='True')
    plt.show()
    print("% of variance explained by the first 2 EOFs : ", int(varfrac[0]*100+varfrac[1]*100))
    i+=1

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) For which variables the percentage of variance explained by the first two spatial structures (EOFs) is maximized ?</b> </p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

<div class="alert alert-danger">
<p><b>Choose the variable for which the EOF analysis of the ensemble will be done.</b></p>
</div>

In [ ]:
data = 
data = np.array(data)
print(data.shape)

In [ ]:
wgts = np.sqrt(np.cos(np.deg2rad(lat)))[:, np.newaxis]
solver = Eof(data, weights=wgts)
eofs = solver.eofs()
pcs = solver.pcs(pcscaling=0)
varfrac = solver.varianceFraction()

print(eofs.shape)
print(pcs.shape)
print(np.max(eofs))
print(np.min(eofs))
print(np.max(pcs))
print(np.min(pcs))

In [ ]:
fig = plt.figure(figsize=(20,10))
fig.suptitle('EOFs', fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(3, 5),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='each', # None/single/each
       cbar_pad=0.2,
       cbar_size='3%',
       label_mode='keep')

for i, ax in enumerate(axgr):
    ax.coastlines()
    ax.set_title('EOF'+str(i+1)+' ('+str(round(varfrac[i]*100,2))+'%)', fontsize=10, loc='center')
    cf = ax.contourf(lon, lat, eofs[i], transform=ccrs.PlateCarree(), cmap='bwr', extend='both')
    c = ax.contour(lon, lat, eofs[i], colors='black', linewidths=1, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(cf)

plt.show()

figname='./figs/eofs_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

fig = plt.figure(figsize=(15,6))
fig.suptitle('PCs', fontsize=16)

plt.subplots_adjust(hspace=0.5, wspace=0.4)

for i in range(0, 15):
        plt.subplot(3, 5, i+1)
        plt.title('PC'+str(i+1))
        plt.axhline(0, color='k', linewidth=0.5)
        plt.bar(members, pcs[:,i], width=0.5)

plt.show()

figname='./figs/pcs_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')
             
fig = plt.figure(figsize=(15, 8))
fig.suptitle('Variance fraction', fontsize=16)

eof_num = range(1, 16)
plt.bar(eof_num, varfrac[0:15], width=0.5)
plt.axhline(0, color='k')
plt.xticks(range(1, 16))
plt.xlabel('EOF #')
plt.ylabel('Variance Fraction')
plt.xlim(1, 15)
plt.ylim(np.min(varfrac), np.max(varfrac)+0.01)
plt.show()

figname='./figs/varfrac_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
clevs = np.linspace(-0.01, 0.01, 11)

fig = plt.figure(figsize=(15, 15))
fig.suptitle('ECMWF ensemble members - EOF analysis (Z500) : '+date_str, fontsize=16)

ax = fig.add_subplot(321, projection=ccrs.PlateCarree())
varfrac1=round(varfrac[0]*100,2)
plt.title('EOF1 ('+str(varfrac1)+'%)', fontsize=10, loc='center')
ax.coastlines()
cf = ax.contourf(lon, lat, eofs[0], clevs, cmap='bwr', extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, eofs[0], clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.7, pad=0.05, extendrect='True')

ax = fig.add_subplot(322)
plt.title('PC1')
plt.xlabel('Member')
pc1=np.asarray(pcs[:,0])
colormat=np.where(pc1>0, 'red','blue')
plt.bar(members, pc1, width=0.5, color=colormat)
plt.axhline(0, color='k')
#ax.set_ylim(-9000, 9000)

ax = fig.add_subplot(323, projection=ccrs.PlateCarree())
varfrac2=round(varfrac[1]*100,2)
plt.title('EOF2 ('+str(varfrac2)+'%)', fontsize=10, loc='center')
ax.coastlines()
cf = ax.contourf(lon, lat, eofs[1], clevs, cmap='bwr', extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, eofs[1], clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.7, pad=0.05, extendrect='True')

ax = fig.add_subplot(324)
plt.title('PC2')
plt.xlabel('Member')
pc2=np.asarray(pcs[:,1])
colormat=np.where(pc2>0, 'red','blue')
plt.bar(members, pc2, width=0.5, color=colormat)
plt.axhline(0, color='k')
#ax.set_ylim(-9000, 9000)

ax = fig.add_subplot(325, projection=ccrs.PlateCarree())
varfrac2=round(varfrac[2]*100,2)
plt.title('EOF2 ('+str(varfrac2)+'%)', fontsize=10, loc='center')
ax.coastlines()
cf = ax.contourf(lon, lat, eofs[2], clevs, cmap='bwr', extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, eofs[2], clevs, colors='black', linewidths=1, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.7, pad=0.05, extendrect='True')

ax = fig.add_subplot(326)
plt.title('PC2')
plt.xlabel('Member')
pc3=np.asarray(pcs[:,2])
colormat=np.where(pc3>0, 'red','blue')
plt.bar(members, pc3, width=0.5, color=colormat)
plt.axhline(0, color='k')
#ax.set_ylim(-9000, 9000)

figname='./figs/eofs_pcs_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

plt.show()

In [ ]:
#levs=clevs_t850
levs=clevs_z
#levs=clevs_mslp

member_number=int(input("Enter a member number (between 1 and 50) : "))

fig = plt.figure(figsize=(10, 10))
fig.suptitle('Reconstruction of ensemble member '+str(member_number)+' with the EOFs', fontsize=14)

ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
plt.title('Z500 field', fontsize=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, data[member_number-1,:,:], levs, transform=ccrs.PlateCarree(), cmap=cmap, extend='both')
c = ax.contour(lon, lat, data[member_number-1,:,:], levs, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.8, pad=0.1, extendrect='True')
cb.set_label('m', fontsize=12)

plt.show()

figname='./figs/reconstruction1'
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
def reconst_eof_pc(eofs,pcs,n):
    reconst = 0
    for i in range(n):
        reconst+=eofs[i]*pcs[member_number-1,i]/wgts
    return(reconst)

fig = plt.figure(figsize=(15, 15))
fig.suptitle('Reconstruction of ensemble member '+str(member_number)+' with the EOFs', fontsize=14)

for i in tqdm(range(1,16)):
    ax = fig.add_subplot(5,3,i, projection=ccrs.PlateCarree())
    plt.title('Reconstruction with '+str(i)+'EOFs', fontsize=10, loc='center')
    plot_background(ax)
    cf = ax.contourf(lon, lat, reconst_eof_pc(eofs,pcs,i)+data.mean(axis=0), levs, transform=ccrs.PlateCarree(), cmap=cmap, extend='both')
    c = ax.contour(lon, lat, reconst_eof_pc(eofs,pcs,i)+data.mean(axis=0), levs, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    #cf = ax.contourf(lon, lat, solver.reconstructedField(i)[member_number-1,:,:]+data.mean(axis=0), levs, transform=ccrs.PlateCarree(), cmap=cmap, extend='both')
    #c = ax.contour(lon, lat, solver.reconstructedField(i)[member_number-1,:,:]+data.mean(axis=0), levs, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    ax.clabel(c, fmt='%4.1i', fontsize=10)

cb = fig.colorbar(cf, orientation='vertical', aspect=65, shrink=0.8, pad=0.1, extendrect='True')
cb.set_label('m', fontsize=12)

plt.show()

figname='./figs/reconstruction2'
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) Is the reconstruction of a ensemble member with 2 EOFs satisfying ?</b></p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

In [ ]:
def plot_phase_space(ax):
    plt.title('Forecast time : '+date_str)
    plt.axhline(0, color='grey', linestyle='--')
    plt.axvline(0, color='grey', linestyle='--')
    plt.xlabel('PC1')
    plt.ylabel('PC2')
    #plt.xlim(-12000, 12000)
    #plt.ylim(-12000, 12000)
    return ax

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111)
fig.suptitle('Ensemble members in PC1 and PC2 phase space (Z500) ', fontsize=14)
plot_phase_space(ax)
plt.scatter(pcs[:,0],pcs[:,1], c = 'g', s=10)

for i,type in enumerate(members_str):
    x = pcs[i,0]
    y = pcs[i,1]
    plt.text(x, y, type, fontsize=8)

plt.show()

figname='./figs/z500_PC_2d_phase space_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) According to this scatterplot in the PC1-PC2 phase space cite 2 members that are very different and two members that are very similar. Verify with the stamp plot above.</b></p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

# Part 7 (advanced) : k-means clustering of the members in PC phase space

In [ ]:
nclusters=3

pcs = np.array([pcs[:,0],pcs[:,1]])
pcs=pcs.transpose()

# Number of clusters
kmeans = KMeans(n_clusters=nclusters)
# Fitting the input data
kmeans = kmeans.fit(pcs)
# Getting the cluster labels
cluster = kmeans.predict(pcs)
# Centroid values
centroids = kmeans.cluster_centers_

print(cluster)
print(cluster.shape)
print(centroids)
print(centroids.shape)

In [ ]:
nc=[list(cluster[:]).count(i) for i in range(nclusters)]
freq=[round(x/cluster.shape[0]*100,2) for x in nc]
print(freq)

In [ ]:
colors=["blue","red", "green"]
color = [colors[num] for num in cluster]
print(cluster)
print(color)

In [ ]:
def plot_cluster_leg(ax):
    patch1 = mpatches.Patch(color=color[0], label='Cluster 1 : '+str(nc[0])+' ('+str(freq[0])+'%)')
    patch2 = mpatches.Patch(color=color[1], label='Cluster 2 : '+str(nc[1])+' ('+str(freq[1])+'%)')
    patch3 = mpatches.Patch(color=color[2], label='Cluster 3 : '+str(nc[2])+' ('+str(freq[2])+'%)')
    all_handles = (patch1, patch2, patch3)
    leg = ax.legend(handles=all_handles, loc='upper right')
    return ax

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111)
fig.suptitle('Ensemble members in PC1 and PC2 phase space - Z500 K-means clustering', fontsize=14)
plot_phase_space(ax)
plt.scatter(pcs[:,0], pcs[:,1], c=color, s=10)
plt.scatter(centroids[:, 0], centroids[:, 1], c=colors, s=2000, alpha=0.5, marker='.');
plot_cluster_leg(ax)

for i,type in enumerate(members_str):
    x = pcs[i,0]
    y = pcs[i,1]
    plt.text(x, y, type, fontsize=8)

plt.show()

figname='./figs/Z500_PC_2d_phase space_clusters_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(15, 10))
fig.suptitle('Geopotential height at 500hPa spaguettis with clustering. Forecast time : '+date_str, fontsize=14)
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
plt.title(str(clevs_spag)+' mgp contour',size=10, loc='center')
plot_background(ax)
for i in range(len(members)):
    c = ax.contour(lon, lat, z500_ens[i,:,:], clevs_spag, colors=color[i], linewidths=1, transform=ccrs.PlateCarree())
plt.show()

figname='./figs/ZT500_ENS_SPAG_cluster_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) Discuss the 3 clusters in terms of positionning of the 500hPa low.</b></p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

In [ ]:
fig = plt.figure(figsize=(15, 10))
fig.suptitle('Mean Sea Level Pressure spaguettis. Forecast time : '+date_str, fontsize=14)
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
plt.title(str(clevs_spag2)+' hPa contour',size=10, loc='center')
plot_background(ax)
for i in range(len(members)):
    c = ax.contour(lon, lat, mslp_ens[i,:,:], clevs_spag2, colors=color[i], linewidths=1, transform=ccrs.PlateCarree())
plt.show()

figname='./figs/MSLP_ENS_SPAG_cluster_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) Discuss the 3 clusters in terms of localization of the low level low pressure system. In with cluster seems to lie the medicane scenario ?</b></p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

In [ ]:
id_cluster1=np.any([cluster==0],axis=0)
id_cluster2=np.any([cluster==1],axis=0)
id_cluster3=np.any([cluster==2],axis=0)

print(cluster)
print(id_cluster1)
print(id_cluster2)
print(id_cluster3)

z_c1 = z500_ens[id_cluster1,:,:].mean(axis=0)
z_c2 = z500_ens[id_cluster2,:,:].mean(axis=0)
z_c3 = z500_ens[id_cluster3,:,:].mean(axis=0)

t500_c1 = t500_ens[id_cluster1,:,:].mean(axis=0)
t500_c2 = t500_ens[id_cluster2,:,:].mean(axis=0)
t500_c3 = t500_ens[id_cluster3,:,:].mean(axis=0)

mslp_c1 = mslp_ens[id_cluster1,:,:].mean(axis=0)
mslp_c2 = mslp_ens[id_cluster2,:,:].mean(axis=0)
mslp_c3 = mslp_ens[id_cluster3,:,:].mean(axis=0)

the_c1 = Thetae_ens[id_cluster1,:,:].mean(axis=0)
the_c2 = Thetae_ens[id_cluster2,:,:].mean(axis=0)
the_c3 = Thetae_ens[id_cluster3,:,:].mean(axis=0)

pv_c1 = pv320_ens[id_cluster1,:,:].mean(axis=0)
pv_c2 = pv320_ens[id_cluster2,:,:].mean(axis=0)
pv_c3 = pv320_ens[id_cluster3,:,:].mean(axis=0)

print(z_c1.shape)
print(mslp_c1.shape)
print(the_c1.shape)
print(pv_c1.shape)

In [ ]:
fig = plt.figure(figsize=(15, 15))
fig.suptitle('Cluster composites - Forecast time : '+date_str, fontsize=14)

ax = fig.add_subplot(331, projection=ccrs.PlateCarree())
plt.title('ZT500 Cluster 1 ('+str(nc[0])+')', color=colors[0], fontsize=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, t500_c1, levels=clevs_t500, cmap=cmap, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, z_c1, levels=clevs_z, colors='black', linewidths=0.5, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('°C', fontsize=12)

ax = fig.add_subplot(332, projection=ccrs.PlateCarree())
plt.title('ZT500 Cluster 2 ('+str(nc[1])+')', color=colors[1], fontsize=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, t500_c2, levels=clevs_t500, cmap=cmap, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, z_c2, levels=clevs_z, colors='black', linewidths=0.5, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('°C', fontsize=12)

ax = fig.add_subplot(333, projection=ccrs.PlateCarree())
plt.title('ZT500 Cluster 3 ('+str(nc[2])+')', color=colors[2], fontsize=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, t500_c3, levels=clevs_t500, cmap=cmap, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, z_c3, levels=clevs_z, colors='black', linewidths=0.5, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('°C', fontsize=12)

ax = fig.add_subplot(334, projection=ccrs.PlateCarree())
plt.title('MSLP-THE850 Cluster 1 ('+str(nc[0])+')', color=colors[0], fontsize=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, the_c1, levels=clevs_the, cmap=cmap, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, mslp_c1, levels=clevs_mslp, colors='black', linewidths=0.5, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('K', fontsize=12)

ax = fig.add_subplot(335, projection=ccrs.PlateCarree())
plt.title('MSLP-THE850 Cluster 2 ('+str(nc[1])+')', color=colors[1], fontsize=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, the_c2, levels=clevs_the, cmap=cmap, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, mslp_c2, levels=clevs_mslp, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('K', fontsize=12)

ax = fig.add_subplot(336, projection=ccrs.PlateCarree())
plt.title('MSLP-THE850 Cluster 3 ('+str(nc[2])+')', color=colors[2], fontsize=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, the_c3, levels=clevs_the, cmap=cmap, extend='both', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, mslp_c3, levels=clevs_mslp, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('K', fontsize=12)

ax = fig.add_subplot(337, projection=ccrs.PlateCarree())
plt.title('PV320K Cluster 1 ('+str(nc[0])+')', color=colors[0], fontsize=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, pv_c1, levels=clevs_pv2, cmap=cmap, extend='max', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, pv_c1, levels=clevs_pv2, colors='black', linewidths=0.5, transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('PVU', fontsize=12)

ax = fig.add_subplot(338, projection=ccrs.PlateCarree())
plt.title('PV320K Cluster 2 ('+str(nc[1])+')', color=colors[1], fontsize=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, pv_c2, levels=clevs_pv2, cmap=cmap, extend='max', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, pv_c2, levels=clevs_pv2, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('PVU', fontsize=12)

ax = fig.add_subplot(339, projection=ccrs.PlateCarree())
plt.title('PV320K Cluster 3 ('+str(nc[2])+')', color=colors[2], fontsize=10, loc='center')
plot_background(ax)
cf = ax.contourf(lon, lat, pv_c3, levels=clevs_pv2, cmap=cmap, extend='max', transform=ccrs.PlateCarree())
c = ax.contour(lon, lat, pv_c3, levels=clevs_pv2, colors='black', linewidths=0.5,transform=ccrs.PlateCarree())
ax.clabel(c, fmt='%4.1i', fontsize=10)
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=1, pad=0.1, extendrect='True')
cb.set_label('PVU', fontsize=12)

plt.show()

figname='./figs/Composites_ZT500_MSLP_THE_PV320_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) Describe for each variable the differences between the 3 clusters. Which cluster is the medicane one ?</b></p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

In [ ]:
plt_title = 'ECMWF ensemble members with clustering - ZT500. Forecast time : '+date_str
projection = ccrs.PlateCarree()

fig = plt.figure(figsize=(20,20))
fig.suptitle(plt_title, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(10, 5),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='1%',
       label_mode='keep')
                   
for i, ax in enumerate(axgr):
    ax.coastlines()
    ax.set_title(str(i+1), color=color[i], fontsize=12)
    p1 = ax.contourf(lon, lat, t500_ens[i,:,:], clevs_t500, transform=ccrs.PlateCarree(), cmap=cmap, extend='both')
    p2 = ax.contour(lon, lat, z500_ens[i,:,:], clevs_z, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(p1)
   
plt.show()

figname='./figs/ZT500_ENS_cluster_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
plt_title = 'ECMWF ensemble members with clustering - MSLP and $\Theta_E$ at 850 hPa. Forecast time : '+date_str
projection = ccrs.PlateCarree()

fig = plt.figure(figsize=(20,20))
fig.suptitle(plt_title, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(10, 5),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='1%',
       label_mode='keep')
                   
for i, ax in enumerate(axgr):
    ax.coastlines()
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    ax.set_title(str(i+1), color=color[i], fontsize=12)
    p1 = ax.contourf(lon, lat, Thetae_ens[i,:,:], clevs_the, transform=ccrs.PlateCarree(), cmap=cmap, extend='both')
    p2 = ax.contour(lon, lat, mslp_ens[i,:,:], clevs_mslp, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(p1)
   
plt.show()

figname='./figs/MSLP_THE_ENS_cluster_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

In [ ]:
plt_title = 'ECMWF ensemble members with clustering - PV320K. Forecast time : '+date_str
projection = ccrs.PlateCarree()

fig = plt.figure(figsize=(20,20))
fig.suptitle(plt_title, fontsize=16)

axgr = AxesGrid(fig, 111, axes_class=axes_class,
       nrows_ncols=(10, 5),
       axes_pad=0.6,
       cbar_location='right',
       cbar_mode='single', # None/single/each
       cbar_pad=0.2,
       cbar_size='1%',
       label_mode='keep')
                   
for i, ax in enumerate(axgr):
    ax.coastlines()
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.5, color='gray', alpha=0.5, linestyle='-')
    ax.set_title(str(i+1), color=color[i], fontsize=12)
    p1 = ax.contourf(lon, lat, pv320_ens[i,:,:], clevs_pv, transform=ccrs.PlateCarree(), cmap=cmap, extend='max')
    p2 = ax.contour(lon, lat, pv320_ens[i,:,:], clevs_pv, colors='black', linewidths=0.2, transform=ccrs.PlateCarree())
    axgr.cbar_axes[i].colorbar(p1)
   
plt.show()

figname='./figs/PV320K_ENS_cluster_'+date_str
fig.savefig(figname+'.png',bbox_inches='tight')

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) Looking at the stamps colored in the corresponding cluster color, is the automatic clustering useful to understand the different forecast scenarios in the ensemble ?</b></p>
</div>

<div class="alert alert-success">
<b>Answer </b>
</div>

# Extra task : projection of the analysis and HRES forecast in the PC phase space (Z500)

<div class="alert alert-danger">
<b>Propose a code to project the analysis and derministic forecast in the PC1-PC2 phase space </b>
    
- Open the analysis data (Z500_September2018_HiRes.nc) and the deterministic forecast data (Z500_fc_20180924.nc) over the same geographical domain and at the same forecast time --> 2 maps.
    
- Compute anomaly for ensemble mean for the analysis and the deterministic forecast and multiply the anomaly by the spatial weights (wgts).

- For the analysis and the deterministic forecast, obtain the PC1 by projecting the anomaly field onto the EOF1. For the analysis and the forecast, obtain the PC2 by projecting the anomaly field onto the EOF1. Basically projecting onto the EOF means doing a scalar product (i.e. sum(weighted anomaly * eof)).
    
- Scatter plot the projected PC1 and PC2 of the analysis and deterministic forecast on the ensemble members scatterplot colored by cluster.
    
- To see to which cluster belong the analysis and deterministic forecast, look at the closest centroid to the projected PCs. You can also try to compute the distance of the PCs from each centroid.
</div>

# Extra task : try another forecast time, EOF analysis with another variable

<div class="alert alert-danger">
<b>Code </b>
</div>